In [27]:
# This needs to be here because by default Jupyter only adds the pwd to sys.path
import os, sys
if os.path.abspath('..') not in sys.path: sys.path.append(os.path.abspath('..'))

import torch
import time
import scipy
import pandas as pd
import numpy as np

from tqdm.auto import tqdm
from pysrc.constants import datapath, N_ITEMS, N_USERS
from torch.utils.data import Dataset
from scipy.sparse import csr_matrix

In [2]:
dataset_indices = {}

### Full Dataset (No Sparsity)

In [3]:
user_ids = []
item_ids = []
values = []
with open(datapath("train.txt")) as file:
    for line in file:
        user_id, items = line.split(maxsplit=1)
        items = [int(id) for id in items.split()]
        user_ids += [int(user_id)] * len(items)
        values += [1] * len(items)
        item_ids += items

train_data = torch.sparse_coo_tensor([user_ids, item_ids], values, (N_USERS, N_ITEMS))
torch.save(train_data, datapath("full_data.pt"))
print(train_data)

dataset_indices["full_data"] = list(range(N_USERS))

tensor(indices=tensor([[    0,     0,     0,  ..., 52642, 52642, 52642],
                       [    0,     1,     2,  ..., 23186, 10690, 10874]]),
       values=tensor([1, 1, 1,  ..., 1, 1, 1]),
       size=(52643, 91599), nnz=2380730, layout=torch.sparse_coo)


### Uniform Degradation

In [4]:
np.random.seed(seed = 0)

for sparsity in [0, .01, .02, .03, .05, .1, .2, .3, .4, .5, .6, .7, .9]:
    user_ids = []
    item_ids = []
    values = []
    with open(datapath("train.txt")) as file:
        for line in file:
            user_id, items = line.split(maxsplit=1)
            items = [int(id) for id in items.split()]
            user_ids += [int(user_id)] * len(items)
            values += [1] * len(items)
            item_ids += items

    indices = np.random.choice(range(len(user_ids)), 
                               size = int(len(user_ids) * (1 - sparsity)), 
                               replace = False)
    user_ids = np.array(user_ids)[indices]
    item_ids = np.array(item_ids)[indices]
    values = np.array(values)[indices]

    train_data = torch.sparse_coo_tensor(np.array([user_ids, item_ids]), values, (N_USERS, N_ITEMS))
    torch.save(train_data, datapath(f"uniform/uniform{int(100*(1 - sparsity))}_data.pt"))

    dataset_indices[f"uniform{int(100*(1 - sparsity))}_data"] = list(range(N_USERS))    

### Top (User) Pruning

In [5]:
train_full = torch.load(datapath("full_data.pt"))

train_dict = {}
with open(datapath("train.txt")) as file:
    for line in file:
        (user_id, items) = line.split(maxsplit=1)
        train_dict[user_id] = items.split()

user_sparsities = {int(k):len(v)/N_ITEMS for k,v in train_dict.items()}
qs = np.array([list(user_sparsities.keys()), list(user_sparsities.values())])

for q in [70, 75, 80, 85, 90, 95]:
    quant = np.quantile(qs[1, :], q * 0.01)
    indices = qs[0, qs[1, :] < quant]

    train_is = train_full.coalesce().indices().numpy()
    train_is = train_is[:, np.isin(train_is[0, :], indices)]

    reindex = 0
    current_value = train_is[0, 0]
    for i in range(train_is.shape[1]):
        if (train_is[0, i] == current_value):
            train_is[0, i] = reindex
        else:
            current_value = train_is[0, i]
            reindex += 1
            train_is[0, i] = reindex

    train_data = torch.sparse_coo_tensor(train_is, np.ones(train_is.shape[1]), (len(indices), N_ITEMS))
    torch.save(train_data, datapath(f"top/top{q}_data.pt"))

    dataset_indices[f"top{q}_data"] = indices    

### Middle Pruning

In [7]:
train_full = torch.load(datapath("full_data.pt"))

train_dict = {}
with open(datapath("train.txt")) as file:
    for line in file:
        (user_id, items) = line.split(maxsplit=1)
        train_dict[user_id] = items.split()

user_sparsities = {int(k):len(v)/N_ITEMS for k,v in train_dict.items()}
qs = np.array([list(user_sparsities.keys()), list(user_sparsities.values())])

for radius in [5, 10, 15, 20, 25, 30]:
    low = np.quantile(qs[1, :], 0.5 - radius * 0.01)
    high = np.quantile(qs[1, :], 0.5 + radius * 0.01)
    indices = qs[0, (qs[1, :] < low) | (qs[1, :] > high)]

    train_is = train_full.coalesce().indices().numpy()
    train_is = train_is[:, np.isin(train_is[0, :], indices)]

    reindex = 0
    current_value = train_is[0, 0]
    for i in range(train_is.shape[1]):
        if (train_is[0, i] == current_value):
            train_is[0, i] = reindex
        else:
            current_value = train_is[0, i]
            reindex += 1
            train_is[0, i] = reindex

    train_data = torch.sparse_coo_tensor(train_is, np.ones(train_is.shape[1]), (len(indices), N_ITEMS))
    torch.save(train_data, datapath(f"middle/middle{radius}_data.pt"))

    dataset_indices[f"middle{radius}_data"] = indices    

### LGR Degredation

Reference: https://www.sciencedirect.com/science/article/pii/S0957417410010985

In [40]:
# Calculate Jaccard matrix between users and determine connected components of user graph

def pairwise_jaccard_sparse(csr, epsilon):
    """
    Reference: https://stackoverflow.com/questions/32805916/compute-jaccard-distances-on-sparse-matrix
    Computes the Jaccard distance between the rows of `csr`,
    smaller than the cut-off distance `epsilon`.
    """
    assert(0 < epsilon < 1)
    csr = csr_matrix(csr).astype(bool).astype(int)

    csr_rownnz = csr.getnnz(axis=1)
    intrsct = csr.dot(csr.T)

    nnz_i = np.repeat(csr_rownnz, intrsct.getnnz(axis=1))
    unions = nnz_i + csr_rownnz[intrsct.indices] - intrsct.data
    dists = 1.0 - intrsct.data / unions

    mask = (dists > 0) & (dists <= epsilon)
    data = dists[mask]
    indices = intrsct.indices[mask]

    rownnz = np.add.reduceat(mask, intrsct.indptr[:-1])
    indptr = np.r_[0, np.cumsum(rownnz)]

    out = csr_matrix((data, indices, indptr), intrsct.shape)
    return out

train_full = torch.load(datapath("full_data.pt")).coalesce()
inds = train_full.indices().numpy()

X = csr_matrix((np.ones(inds.shape[1]), (inds[0, :], inds[1, :])), shape = (N_USERS, N_ITEMS))
similarities = pairwise_jaccard_sparse(X, 0.87)

N, components = scipy.sparse.csgraph.connected_components(similarities)

In [ ]:
# TODO